In [6]:
import pandas as pd
import requests 
from urllib.parse import urlencode
import numpy as np
import pingouin as pg
import matplotlib.pyplot as plt
import plotly.express as px
from scipy import stats
import pandahouse as ph
import nbformat

# Задача 1.

В ходе тестирования гипотезы целевой группе была предложена новая механика оплаты услуг на сайте, у контрольной группы оставалась базовая механика

Задача:

Проанализировать итоги эксперимента и сделать вывод, стоит ли запускать новую механику оплаты на всех пользователей

In [7]:
def upload_data(public_key, name_csv):
    base_url = 'https://cloud-api.yandex.net/v1/disk/public/resources/download?'
    final_url = base_url + urlencode(dict(public_key=public_key))  
    response = requests.get(final_url)
    download_url = response.json()['href']
    download_response = requests.get(download_url)
    with open(name_csv, 'wb') as f:   
        f.write(download_response.content)

In [8]:
upload_data('https://disk.yandex.ru/d/UhyYx41rTt3clQ','groups.csv')
upload_data('https://disk.yandex.ru/d/5Kxrz02m3IBUwQ','group_add.csv')
upload_data('https://disk.yandex.ru/d/Tbs44Bm6H_FwFQ', 'active_studs.csv')
upload_data('https://disk.yandex.ru/d/pH1q-VqcxXjsVA','checks.csv')

In [9]:
groups = pd.read_csv('groups.csv', sep=';')
group_add = pd.read_csv('group_add.csv')
active_studs = pd.read_csv('active_studs.csv')
checks = pd.read_csv('checks.csv', sep=';')

- groups.csv – файл с информацией о принадлежности пользователя к контрольной или экспериментальной группе (А – контроль, B – целевая группа) 
- groups_add.csv – дополнительный файл с пользователями
- active_studs.csv – файл с информацией о пользователях, которые зашли на платформу в дни проведения эксперимента. 
- checks.csv – файл с информацией об оплатах пользователей в дни проведения эксперимента.

In [10]:
df = pd.concat([groups, group_add], axis=0)

Объединяем вертикально groups и group_add для получения итоговой таблицы с контрольной и тестовой группами

In [11]:
df.shape

(74576, 2)

In [12]:
df.drop_duplicates().head()

,id,grp
0,1489,B
1,1627,A
2,1768,B
3,1783,B
4,1794,A


Проверяем отсутствие дубликатов

In [13]:
active_studs.head()

,student_id
0,581585
1,5723133
2,3276743
3,4238589
4,4475369


In [14]:
active_studs.rename(columns={'student_id':'id'}, inplace=True) 

Переименовываем колонку для удобства

In [15]:
active = active_studs.merge(df, how='left', on='id')

Получаем датафрейм с пользователями контрольной и тестовой групп, которые заходили на платформу в дни проведения эксперимента

In [16]:
active.shape

(8341, 2)

In [17]:
active.drop_duplicates().head()

,id,grp
0,581585,A
1,5723133,A
2,3276743,B
3,4238589,A
4,4475369,B


In [18]:
active.value_counts('grp')

grp
B    6803
A    1538
Name: count, dtype: int64

Выборка контрольной группы меньше, чем тестовой

In [19]:
checks.head()

,student_id,rev
0,1627,990.0
1,3185,690.0
2,25973,690.0
3,26280,690.0
4,100300,990.0


In [20]:
checks.rename(columns={'student_id':'id'}, inplace=True)

In [21]:
buyers = checks.merge(active, how='left', on='id')

In [22]:
buyers.head()

,id,rev,grp
0,1627,990.0,A
1,3185,690.0,NaN
2,25973,690.0,NaN
3,26280,690.0,NaN
4,100300,990.0,B


Получаем информацию о пользователях, которые проводили оплату в дни эксперимента

In [23]:
buyers = buyers.dropna()

Удалим пользователей, неучаствующих в эксперименте (либо в данных отсутствует информация о принадлежности пользователя к конкретной группе)

In [24]:
buyers.value_counts('rev')

rev
1900.0000    93
290.0000     70
1900.0001    35
199.0000     32
840.0000     24
990.0000     22
690.0000     17
1140.0000    11
580.0000     11
590.0000     11
1530.0000     8
398.0000      7
870.0000      6
3660.0000     5
2910.0000     4
2890.0000     4
489.0000      3
2099.0000     3
1160.0000     3
1980.0000     3
2220.0000     2
2820.0000     2
1729.0000     1
1388.0000     1
1339.0000     1
1180.0000     1
1050.0000     1
597.0000      1
880.0000      1
1039.0000     1
796.0000      1
2190.0000     1
1830.0000     1
1968.0000     1
2900.0000     1
3040.0000     1
3260.0000     1
4650.0000     1
Name: count, dtype: int64

В глаза бросается количество чеков стоимостью 1900 и 290, возможно, это связано со спефицикой продукта

In [25]:
buyers['rev'] = buyers.rev.astype('int64')

Для удобства можно убрать десятичную дробь в колонке с информацией об оплатах, она здесь ни к чему

In [26]:
buyers.value_counts('grp')

grp
B    314
A     78
Name: count, dtype: int64

В контрольной выборке 78 пользователей,  в тестовой - 314

Выбранная гипотеза предполагает связь новой механики оплаты услуг на сайте с увеличением выручки

Метрики, с помощью которых можно сравнить две группы:

1. Сравнение среднего чека в контрольной и тестовой группах
2. Конверсия посещения в покупку

# 1. Начнём с рассмотрения первой метрики и сравним средний чек в контрольной и тестовой группе

Я предлагаю сделать сравнение средних с помощью т-критерия, так как у нас есть категориальная и количественная переменная, а выборки независимы.

Размеры выборок (>30) позволяют нам это сделать

Требования к проведению т-теста:
1. Требование гомогенности дисперсий
2. Требование к нормальности распределения средних

Нулевая гипотеза:
Средний чек для контрольной и тестовой групп в генеральной совокупности не отличается

Альтернативная гипотеза: Средний чек для контрольной и тестовой групп в генеральной совокупности отличается

In [27]:
pg.homoscedasticity(buyers, dv='rev',group='grp', method='levene',alpha=0.05)

,W,pval,equal_var
levene,3.075681,0.080257,True


Проверяем однородность дисперсий (дисперсии однородны)

In [28]:
px.histogram(buyers[buyers['grp']=='A'], x='rev')

In [29]:
px.histogram(buyers[buyers['grp']=='B'], x='rev')

Как видно по графикам, данные в выборках распределены ненормально, можно попробовать логарифмирование:

In [30]:
buyers['log_rev'] = np.log(buyers.rev)


In [31]:
px.histogram(buyers[buyers['grp']=='A'], x='log_rev')

In [32]:
px.histogram(buyers[buyers['grp']=='B'], x='log_rev')

Ситуация особо не изменилась

In [33]:
pg.normality(data=buyers, dv="rev", group="grp", method="normaltest")

,W,pval,normal
grp,,,
A,30.378420,2.531687e-07,False
B,6.795664,3.344570e-02,False


In [34]:
pg.normality(data=buyers, dv="log_rev", group="grp", method="normaltest")

,W,pval,normal
grp,,,
A,7.991540,1.839328e-02,False
B,85.325127,2.964040e-19,False


Подтверждаем наблюдения с графиков с помощью теста на нормальность

In [35]:
A = buyers[buyers['grp']=='A']['rev']

In [36]:
B = buyers[buyers['grp']=='B']['rev']

In [37]:
stats.ttest_ind(A, B, equal_var = True, alternative = 'two-sided')

TtestResult(statistic=np.float64(-3.133779424369783), pvalue=np.float64(0.0018562030244934635), df=np.float64(390.0))

In [38]:
buyers.groupby('grp', as_index = False).agg({'rev':'mean'}).rename(columns={'rev':'mean_rev'})

,grp,mean_rev
0,A,933.589744
1,B,1257.878981


p-value < 0.05, отклоняем нулевую гипотезу

Средний чек для контрольной и тестовой групп значимо отличается, в тестовой группе(В) средний чек выше.

# 2. Посмотрим, отличается ли для тестовой и контрольной групп конверсия посещения сайта в покупку

In [39]:
active.isna().sum()

id     0
grp    0
dtype: int64

In [40]:
buyers.isna().sum()

id         0
rev        0
grp        0
log_rev    0
dtype: int64

In [59]:
conversion = active.merge(buyers, how='left', on=['id','grp'])

In [60]:
conversion.head()

,id,grp,rev,log_rev
0,581585,A,NaN,NaN
1,5723133,A,NaN,NaN
2,3276743,B,NaN,NaN
3,4238589,A,NaN,NaN
4,4475369,B,NaN,NaN


In [42]:
conversion.drop(columns='log_rev', inplace = True)

In [43]:
conversion = conversion.fillna('0')

In [44]:
conversion['rev'] = conversion['rev'].apply('int64')

In [45]:
conversion['purchase'] = (conversion.rev > 0)*1

In [46]:
table = conversion.groupby(['grp','purchase'], as_index = False).agg({'id':'count'})

In [47]:
table

,grp,purchase,id
0,A,0,1460
1,A,1,78
2,B,0,6489
3,B,1,314


In [48]:
table = pd.pivot_table(table, values='id', columns = 'purchase', index='grp')

In [49]:
table

purchase,0,1
grp,,
A,1460.0,78.0
B,6489.0,314.0


В данном случае предлагаю использовать критерий хи-квадрат:
1. Наблюдения независимы
2. Количество наблюдений > 5
3. Исследуется взаимосвязь номинативных переменных

Нулевая гипотеза: 
Конверсия контрольной группы равна конверсии тестовой группы

Альтернативная гипотеза:
Конверсия контрольной группы отличается от конверсии тестовой группы

In [50]:
stats.chi2_contingency(table, correction=False)

Chi2ContingencyResult(statistic=np.float64(0.5821513741106591), pvalue=np.float64(0.44547028437158964), dof=1, expected_freq=array([[1465.71897854,   72.28102146],
       [6483.28102146,  319.71897854]]))

p-value > 0.05, принимаем нулевую гипотезу, конверсии для обеих групп одинаковы

In [51]:
table['conversion'] = round(100*table[1] / table[0],2)

In [52]:
table

purchase,0,1,conversion
grp,,,
A,1460.0,78.0,5.34
B,6489.0,314.0,4.84


В выборочных данных конверсия тестовой группы даже ниже, чем у контрольной. Но,  как мы выяснили с помощью критерия хи-квадрат, данное отличие не является статистически значимым

# Выводы
Был проведён анализ двух метрик: средний чек и конверсия на этапе оплаты

1. Средний чек для контрольной и тестовой групп значимо отличается, в тестовой группе(В) средний чек выше. На это определённо стоит обратить внимание, но я не думаю, что это напрямую связано с механикой оплаты, так как стоимость покупки формируется на предыдущих этапах, например, в корзине пользователю предлагают приобрести дополнительные товары

2. Конверсия на этапе оплаты для тестовой и контрольной групп значимо не отличается. На мой взгляд, эта метрика более показательная для поставленной задачи. 

Возможно, если бы у меня было больше информации о том, какие конкретно изменения произошли в механике оплаты, вывод был бы более точным

На данный момент,  я считаю, что новую механику оплаты не стоит вводить для всех пользователей. Нужно провести дальнейший анализ и понять, почему в тестовой группе средний чек оказался выше.

# Задача 2

Провести анализ студентов на образовательной платформе.

2.1  Образовательные курсы состоят из различных уроков, каждый из которых состоит из нескольких маленьких заданий. Каждое такое маленькое задание называется "горошиной". Назовём очень усердным учеником того пользователя, который хотя бы раз за текущий месяц правильно решил 20 горошин. 
Задача: Необходимо написать оптимальный запрос, который даст информацию о количестве очень усердных студентов.Под усердным студентом мы понимаем студента, который правильно решил 20 задач за текущий месяц.

2.2  Образовательная платформа предлагает пройти студентам курсы по модели trial: студент может решить бесплатно лишь 30 горошин в день. Для неограниченного количества заданий в определенной дисциплине студенту необходимо приобрести полный доступ. Команда провела эксперимент, где был протестирован новый экран оплаты.

Необходимо в одном запросе выгрузить следующую информацию о группах пользователей:

- ARPU 
- ARPAU 
- CR в покупку 
- СR активного пользователя в покупку 
- CR пользователя из активности по математике (subject = ’math’) в покупку курса по математике

In [53]:
connection = dict(database='default',
                  host='https://clickhouse.lab.karpov.courses',
                  user='student',
                  password='dpo_python_2020')

Задача 2.1

In [54]:
query1 = """
SELECT COUNT(DISTINCT st_id) as hard_students 
FROM
(
SELECT st_id, SUM(correct) AS sum_correct, subject 
FROM default.peas 
WHERE correct == 1
GROUP BY st_id, subject
HAVING SUM(correct) >= 20
) AS r
"""

hard_students = ph.read_clickhouse(query1, connection=connection)
hard_students


,hard_students
0,78


Данные были только за октябрь 2021 года, поэтому ограничение по времени писать не нужно

Задание 2.2

In [55]:
query2 = """
SELECT round((rev.revenue/all_st.all_st_count),2) as ARPU, round((rev.revenue/act_st.active_st_count),2) as ARPAU,
round((100*rev.paying_st/all_st.all_st_count),2) as CR_all_st, round((100*rev.paying_st/act_st.active_st_count),2) as CR_act_st, 
round((100*paying_st_math.paying_st_math_count/act_st_math.act_st_math_count),2) as CR_math_st,all_st.test_grp test_grp

FROM 
(select count(distinct st_id) as all_st_count, test_grp from default.studs
group by test_grp) as all_st

JOIN 

(SELECT count(distinct st_id) as active_st_count, test_grp from
(SELECT st_id, sum(correct) as sum_correct, test_grp, subject FROM
(select a.st_id, a.correct, a.subject, b.test_grp from default.peas as a
left join default.studs as b on a.st_id = b.st_id) as r1
WHERE correct == 1
GROUP BY test_grp, subject, st_id
HAVING sum(correct) >= 10) as r2
group by test_grp)
as act_st on all_st.test_grp = act_st.test_grp

JOIN 

(select sum(a.money) as revenue, count(distinct b.st_id) as paying_st, b.test_grp
from default.final_project_check as a
left join default.studs as b on a.st_id = b.st_id
group by b.test_grp) as rev on rev.test_grp = all_st.test_grp

JOIN 
(SELECT COUNT(DISTINCT st_id) as act_st_math_count, test_grp FROM
(SELECT st_id, sum(correct) as sum_correct, test_grp, subject FROM
(select a.st_id, a.correct, a.subject, b.test_grp from default.peas as a
left join default.studs as b on a.st_id = b.st_id) as r1
WHERE correct == 1 and subject == 'Math'
GROUP BY test_grp, subject, st_id
HAVING sum(correct) >= 2) as act_st_math
group by test_grp) AS act_st_math on rev.test_grp = act_st_math.test_grp

JOIN 
(select COUNT(DISTINCT a.st_id) as paying_st_math_count, b.test_grp from default.final_project_check as a
LEFT JOIN default.studs as b on a.st_id = b.st_id
WHERE subject == 'Math'
GROUP BY b.test_grp) AS paying_st_math ON rev.test_grp = paying_st_math.test_grp
"""

metrics = ph.read_clickhouse(query2, connection=connection)
metrics

,ARPU,ARPAU,CR_all_st,CR_act_st,CR_math_st,test_grp
0,4540.98,11080.0,4.92,12.00,16.33,control
1,11508.47,35000.0,10.85,32.99,38.10,pilot


Для решения этой задачи я рассчитала отдельно 5 таблиц с целью получения всех необходимых показателей (все студенты, активные студенты, выручка и т.д.), затем соединила по критерию тестовой и контрольной групп и получила необходимые метрики

Все три конверсии представлены в процентах

На данный момент по таблице видно, что в тестовой группе(pilot) все показатели выше (лучше), чем в контрольной. Осталось проверить, являются ли эти изменения статистически значимыми

Дополнительно были реализованы: 
1. Функция, которая будет автоматически подгружать информацию из дополнительного файла groups_add.csv и на основании дополнительных параметров пересчитывать метрики(средний чек, конверсия) и проводить статистические тесты 

2. Функция, которая будет строить графики по получаемым метрикам.

In [62]:
def add_groups(link):
    base_url = 'https://cloud-api.yandex.net/v1/disk/public/resources/download?'
    public_key = link  

    final_url = base_url + urlencode(dict(public_key=public_key))
    response = requests.get(final_url)
    download_url = response.json()['href']

    download_response = requests.get(download_url)
    with open('groups_add.csv', 'wb') as f:   
        f.write(download_response.content)
    groups_add = pd.read_csv('groups_add.csv')
    df = pd.concat([groups, groups_add], axis=0)
    df.drop_duplicates()
    active =active_studs.merge(df, how='left', on='id')
    active.drop_duplicates()
    buyers = checks.merge(active, how='left', on='id')
    A = buyers[buyers['grp']=='A']['rev']
    B = buyers[buyers['grp']=='B']['rev']
    winner = buyers.groupby('grp', as_index = False).agg({'rev':'mean'}).rename(columns={'rev':'mean_rev'}) \
    .sort_values('mean_rev', ascending = False).grp.iloc[0]
    if float(pg.homoscedasticity(buyers, dv='rev',group='grp', method='levene',alpha=0.05).pval) > 0.05:
        if stats.ttest_ind(A, B, equal_var = True, alternative = 'two-sided').pvalue > 0.05:
            average_check = 'Средний чек для контрольной и тестовой групп значимо не отличается'
        else:
            average_check = 'Средний чек для контрольной и тестовой групп значимо отличается, средний чек выше у группы ' + winner+'.'
    else: 
        if stats.ttest_ind(A, B, equal_var = False, alternative = 'two-sided').pvalue > 0.05:
                average_check = 'Средний чек для контрольной и тестовой групп значимо не отличается'
        else:
            average_check = 'Средний чек для контрольной и тестовой групп значимо отличается, средний чек выше у группы ' + winner +'.'
            
    
    conversion = active.merge(buyers, how='left', on=['id','grp'])
    conversion = conversion.fillna('0')
    conversion['rev'] = conversion['rev'].apply('int64')
    conversion['purchase'] = (conversion.rev > 0)*1
    table = conversion.groupby(['grp','purchase'], as_index = False).agg({'id':'count'})
    table = pd.pivot_table(table, values='id', columns = 'purchase', index='grp')
    if stats.chi2_contingency(table, correction=False)[1] > 0.05:
        conversion_test = 'Конверсия на этапе оплаты для тестовой и контрольной групп значимо не отличается'
    else: 
        conversion_test = 'Конверсия на этапе оплаты для тестовой и контрольной групп значимо отличается, конверсия выше у группы ' + winner_conv + '.'
    table['conversion'] = round(100*table[1] / table[0],2)
    winner_conv = table.reset_index().sort_values('conversion', ascending = False).grp.iloc[0]
    
    return average_check + ' '+ conversion_test

In [63]:
add_groups('https://disk.yandex.ru/d/5Kxrz02m3IBUwQ')

C:\Users\User\AppData\Local\Temp\ipykernel_8264\317923717.py:22: FutureWarning:

Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead



'Средний чек для контрольной и тестовой групп значимо отличается, средний чек выше у группы B. Конверсия на этапе оплаты для тестовой и контрольной групп значимо не отличается'

Функция add_groups для поиска статистически значимых отличий в среднем чеке и конверсии для контрольной и тестовой групп

In [66]:
def graphs(df):
    avg_check = px.box(buyers, x="grp", y="rev", title='Стоимость покупки в контрольной A и тестовой B группах', labels={'grp': 'Группа', 'rev': 'Стоимость покупки'})
    a_distribution = px.histogram(buyers[buyers['grp'] == 'A'], x='rev', title = 'Распределение данных о стоимости покупок для контрольной группы А', labels = {'rev':'чек'})
    b_distribution = px.histogram(buyers[buyers['grp'] == 'B'], x='rev', title = 'Распределение данных о стоимости покупок для контрольной группы B', labels = {'rev':'чек'})

    return a_distribution.show(), b_distribution.show(), avg_check.show()

Функция graphs отображает графики распределения и диаграмму размаха стоимости покупок для групп А и В

In [65]:
graphs(buyers)

(None, None, None)